<div style="
    border: 2px solid #4CAF50; 
    padding: 15px; 
    background-color: #f4f4f4; 
    border-radius: 10px; 
    align-items: center;">

<h1 style="margin: 0; color: #4CAF50;">Neural Networks: Trainieren von Neuronalen Netzwerken</h1>
<h2 style="margin: 5px 0; color: #555;">DSAI</h2>
<h3 style="margin: 5px 0; color: #555;">Jakob Eggl</h3>

<div style="flex-shrink: 0;">
    <img src="https://www.htl-grieskirchen.at/wp/wp-content/uploads/2022/11/logo_bildschirm-1024x503.png" alt="Logo" style="width: 250px; height: auto;"/>
</div>
<p1> © 2025/26 Jakob Eggl. Nutzung oder Verbreitung nur mit ausdrücklicher Genehmigung des Autors.</p1>
</div>
<div style="flex: 1;">
</div>   

In diesem Notebook wollen wir nun besprechen, wie ein Neuronales Netzwerk lernt.

Dazu können wir uns zuerst einmal fragen, was eigentlich **Lernen** bedeutet?

> A computer program is said to **learn** from **experience E**
with respect to some class of **tasks T** and
performance **measure P**, if its performance at tasks in T,
as measured by P, improves with experience E.

(Mitchell 1997)

Was bedeutet das in unserem Fall?

* Wir sagen, dass unser Modell lernt, falls es bei einer gegebenen Aufgabe *besser wird*:
    * Was ist die Aufgabe bei uns?
    * Wie messen wir, ob es besser geworden ist?

## Die Problemstellung

### Die Ausgangslage

Wir betrachten jetzt nochmal im Detail unsere Problemstellung.

Wir haben Daten für ein Supervised Machine Learning Setting, sprich wir haben eine Menge $\mathcal Z$, welche aus den Paaren $(X_i, y_i)$ besteht, wobei $X_i$ der Feature Vektor für den $i$-ten Datenpunkt und $y_i$ das dazugehörige $i$-te Label (Regression oder Klassifikation) ist.

Ziel ist es, eine Funtion $f$ zu finden, welche uns diesen Zusammenhang abbildet.

![Neural_Network_Function_Approximation](../resources/NN_Function_Approximation.png)

(von https://stackoverflow.com/questions/13897316/approximating-the-sine-function-with-a-neural-network)

Was wird in unserem Fall in Zukunft die Funktion $f$ sein?

### Der Task eines Neuronalen Netzwerkes

Was ist nun die Aufgabe von unserem Neuronalen Netz?

Wir wollen die gegebenen Datenpaare gut mit unserem neuronalen Netz abbilden können. Sprich unser Neuronales Netzwerk soll überall ähnliche (bzw. die gleichen Werte) ausspucken.

Wie können wir das überprüfen?

Händisch können wir natürlich die Ergebnisse mit den originalen Ergebnisse gut vergleichen und die Qualität beurteilen. Dies ist aber mühsam, somit stellt sich uns die Frage, wie wir sonst vorgehen können? Für eindimensionale Daten können wir auch eine Kurve (so wie oben) plotten, auch das ist einfach. Dies ist aber auch in der Praxis meistens nicht der Fall.

Um den Fehler quantifizieren zu können, widmen wir uns jetzt den sogenannten **Loss**-Funktionen. Sie sind uns schon von der linearen (bzw. logistischen) Regression bekannt und geben uns den Fehler, den das Modell macht.

### Die Loss Funktion

Die Loss Funktion gibt uns den Fehler zurück, den das Netzwerk aktuell für einen Input $X_i$ im Vergleich zum Ziel Output $y_i$, macht.

Wir schreiben von nun an für den Loss:
$$L(\hat{y}_i, y_i)=L(\hat f(X_i), y_i),$$

wobei sämtliche Größen mit $\mathbf{\hat{}}$-Symbol immer als die von uns approximierten Größen (Predictions) bezeichnet werden.

Kommen wir nun zu den gängigsten Loss-Funktionen. Wir starten mit den Loss-Funktionen für die **Regression**. Diese sind die gleichen Loss-Funktionen wie für die lineare (logistische) Regression und werden hier kurz aufgezählt.

#### Gängige Loss-Funktionen für Regression

**MSE:** Mean-Squared Error: $$L_{\textrm{MSE}}(\hat{y}, y)=\frac{1}{n}\sum_{i=1}^{n}(\hat{y}_i - y_i)^2,$$

mit $n$ als Anzahl der Datenpunkte.

**MAE:** Mean Absolute Error: $$L_{\mathrm{MAE}}(\hat{y}, y)=\frac{1}{n}\sum_{i=1}^{n}\lvert \hat{y}_i - y_i\rvert.$$

**Hinweis:** Nachdem wir hier die Regressionsloss-Funktionen betrachten, nehmen wir an, dass es jeweils nur ein Output Neuron gibt. Ansonsten müssten wir die Loss-Funktionen leicht adaptieren hier.

#### Gängige Loss-Funktionen für Klassifikation

Hier ist der Output immer ein Vektor, welchen wir mit dem Zielvektor vergleichen wollen. Siehe folgendes Beispiel, wo das Label nun offensichtlich die Klasse **Auto** ist. Mit einem OneHot-Encoder erhalten wir das gewünschte Format für die Loss-Funktion, sprich $\text{Auto}=[1,0,0,0]$.

![Loss_Example_Car](../resources/loss_car_example.jpeg)

(von https://www.shopdev.co/blog/cross-entropy-in-machine-learning)

![Loss_Example_Car](../resources/loss_car_example_2.jpeg)

(von https://www.shopdev.co/blog/cross-entropy-in-machine-learning)

**Categorical Cross-Entropy Loss**: Berchnet, wie sehr sich die Verteilungen $y$ und $\hat{y}$ ähnlich sind. Er ist folgendermaßen definiert:
$$L_{\mathrm{CE}}(\hat{y}, y)=-\frac{1}{n}\sum_{i=1}^{n}\sum_{k=1}^{K}y_{ik}\log(\hat{y}_{ik}),$$

wobei $K$ die Anzahl der Klassen ist, also im obigen Beispiel zum Beispiel 4 und $n$ wieder die Anzahl der Datenpunkte.

**Binary Cross-Entropy Loss** Spezialfall für nur 2 Klassen vom Categorical Cross-Entropy Loss ($K=2$). Es gilt automatisch, weil die Summe vom Vektor $1$ ergeben muss (repräsentiert ja die Wahrscheinlichkeiten bzw. Predictions), dass $y_{i2} = 1-y_{i1}$. Somit ergibt sich für den Loss:
$$L_{\mathrm{BCE}}(\hat{y}, y)=-\frac{1}{n}\sum_{i=1}^{n}[y_i \log (\hat{y}_i)+ (1-y_i)\log(1-\hat{y}_i)].$$

**Wichtig:** Hier ist $y$ die Ziel-Verteilung und $\hat y$ die Verteilung der Vorhersage (Prediction). 

> **Übung:** Warum sind diese beiden Loss-Funktionen mit einem negativen Vorzeichen behaftet?

> **Übung:** Wann ist der Loss 0 für den *Binary Cross-Entropy Loss*? Wann für den *Categorical Cross-Entropy Loss*?

**>Übung:** Der Logarithmus $\log(x)$ ist für $x\leq 0$ nicht definiert. Wieso macht uns das hier keine Probleme? *Tipp:* Softmax.

### Zusammenfassung vom Task eines Neuronalen Netzwerkes:

Folgende Punkte fassen nun unsere Ausgangssituation zusammen.

* Wir haben ein Neuronales Netzwerk (beliebig "komplex", beliebige Aktivierungsfunktionen etc.). Dieses stellt unsere Funktion $\hat{f}$ dar
* Wir haben die Datenpaare mit Features $X_i$ und Label (Wert oder Klasse) $y_i$
* Wir haben die Daten in Trainings- und Testsplit aufgeteilt (zBsp im Verhältnis: 80/20)
* Wir haben eine Loss-Funktion $L(\hat{f}(X_i), \mathbf y_i)$ definiert
* Diese Loss Funktion wollen wir auf den Trainingsdaten minimieren.

**Wichtig:** Auch wenn wir den Loss auf den Trainingsdaten minimieren, wollen wir in Summe natürlich dann ein Modell, bei dem der Loss am *Testset* niedrig ist. (Nur so können wir überprüfen, dass wir nicht overfitten.)

**Hinweis:** Wir können im Anschluss dann auch noch Metriken berechnen wie zum Beispiel Accuracy, Confusion Matrix, Anzahl der True-Positive samples, etc. Solche Metriken sind für uns Anwender oft von großem Interesse, da sie meistens dann die Brücke bilden zu den Praxisproblemen (zbsp.: Anzahl der Fehlklassifikationen bei Krebs Früherkennung usw.). Der Loss selber wird jedoch benötigt zum Optimieren (=Minimieren), also das Modell soll optimiert werden, sodass wir einen kleinen Loss (Fehler) haben. Der Grund dafür ist, dass wir unsere Loss-Funktion ableiten müssen. Jene Funktionen, die uns die Metriken liefern sind oft nicht differenzierbar.

**Hinweis:** Wir können uns auch eine eigene Loss-Funktionen basteln, welche auf besondere Wünsche/Anforderungen abgestimmt ist. Man muss dabei aber auf ein paar Dinge acht geben (werden wir uns eventuell zu einem späteren Zeitpunkt ansehen).

---

### Finden eines Minimums (der Lossfunktion)

Nun bleibt nur mehr die Frage, wie wir das Minimum einer Funktion finden?

**Erinnerung:** Wie finden wir das Minimum einer Funktion?

> **Übung:** Wie müssen die Parameter $w_1, w_2 \in \mathbb R$ gewählt werden, um ein die Funktion $$L(w_1, w_2)=(4w_1+7w_2-20)^2$$ zu minimieren?

> **Übung:** Wie müssen die Parameter $w_1, w_2, w_3 \in \mathbb R$ gewählt werden, um ein die Funktion $$L(w_1, w_2, w_3)=\max (0, 2w_1+3w_2-4w_3)^2$$ zu minimieren?

> **Übung:** Wie muss der Parameter $w_1 \in \mathbb R$ gewählt werden, um ein die Funktion $$L(w_1)=\lvert w_1 - e^{-w_1} \rvert$$ zu minimieren?

**Allgemein gilt natürlich:**

Bei einem Extrempunkt ist die Ableitung $0$. Somit berechnen wir hier einfach die (partielle) Ableitung und wir erhalten alle Kandidaten. Im Anschluss können wir dann leicht prüfen, ob es ein Maximum oder Minimum ist (im 1d mit der 2. Ableitung, ansonsten zum Beispiel auch mit Einsetzen und Vergleichen der Werte).

Aber können wir immer so leicht die Ableitung $0$ setzen?

**Nein**, weil unsere Funktionen (neuronalen Netze) sind sehr kompliziert (und besitzen viele Parameter), weswegen das Berechnen der Nullstelle der Ableitung sich als schwierig/unmöglich gestaltet. Dies ist zum Beispiel beim letzten der drei Beispiele auch schon sehr schwer/unmöglich. 

Tatsächlich ist dies in der Praxis oft (bei Neuronalen Netzen quasi immer) der Fall, wie die folgenden beiden Grafiken zeigen. 

![Loss_Landscape](../resources/Loss_Landscape.png)

(von https://www.cs.umd.edu/~tomg/projects/landscapes/)

![Loss_Landscape_incl_Path](../resources/Loss_Landscape_Path.jpeg)

(von https://discuss.pytorch.org/t/looking-for-the-lost-function-generating-the-lost-landscape-shown-in-this-article-on-sceince/130626)

Wie wir oben sehen (diese Art von Bilder nennt man *Loss Landscape*), haben wir bei solchen Funktionen ein riesen Problem, das **globale** Minimum zu finden. Wir haben aber auch schon Probleme überhaupt ein Minimum zu berechnen.

**Hinweis:** Obige Modelle haben nur 2 Parameter (zum Beispiel hat die Funktion $f(x)=kx+d$ auch nur 2 Parameter ($k, d$)), welche auf den $x$- und $y$-Achsen positioniert sind. Auf der $z$-Achse wird der Loss aufgetragen.

Im Vergleich: Ein Language Model von Meta (llama4) hat etwa 405B Parameter, dass sind 405 Milliarden solcher Parameter, wo wir das gemeinsame Optimimum finden wollen.

Was ist, wenn wir das Minimum nicht analytisch berechnen können?

In so einem Fall müssen wir uns einer numerischen/iterativen Methode bedienen, um unser Minimum zu finden. Eine bekannte Version davon ist **Gradient Descent**.

---

## Gradient Descent

![Gradien_Descent](../resources/gradient_descent_mountain.png)

(von https://ryanwingate.com/intro-to-machine-learning/deep-learning-with-pytorch/training-neural-networks-with-pytorch/)

Zuerst wollen wir uns einmal ansehen, woher der Begriff **Gradient Descent** eigentlich kommt.

**Gradient:** Eine mehrdimensionale Ableitung wird *Gradient* genannt. Wir verwenden dafür das Symbol $\nabla$. Zum Beispiel hat die Funktion $f(x,y)=x^2+y^3+xy$ als Gradient den folgenden *Vektor*: $\nabla f(x,y) = (\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}) = (2x+y,3y^2+x)$. Im Punkt $(1,1)$ ist somit der Gradient $\nabla f(1,1) = (3,4)$. Er gibt also an, in welche Richtung die Funktion wie stark ansteigt (oder fällt bei einem negativen Vorzeichen). In unserem Fall steigt die Funktion am Punkt $(1,1)$ mit Steigung $3$ in Richtung $x$ und Steigung $4$ in Richtung $y$.

**Descent:** Englisches Wort für *Abstieg*.

Wir wollen also die besten Parameter für unser Modell finden, indem wir bei der Loss-Landscape immer in jene Richtung gehen, wo es am steilsten nach unten geht! Dabei wollen wir die Information vom Gradienten verwenden, also von der (mehrdimensionalen) Ableitung.

> **Übung:** Wie können wir diese Richtung finden?

Der Gradient zeigt immer in jene Richtung, wo der Anstieg am größten ist. Somit geht es in die genau andere Richtung am steilsten nach unten!

Somit gehen wir bei dem Gradient Descent Algorithmus immer iterativ (also Schritt für Schritt) in die negative Richtung vom Gradienten an dem jeweiligen Punkt. Wie groß der Schritt in die jeweilige Richtung ist, wird über die sogenannte **Learning-Rate** $\eta\in \mathbb R$ definiert. Sie ist in diesem Fall die Schrittweite und ein Hyperparameter, der vorher festgelegt werden muss. Wir formulieren nun die *Update-Rule*, also die Formel, mit der wir unsere Parameter $w$ vom neuronalen Netzwerk anpassen.

$$w_t = w_{t-1} - \eta \cdot \nabla L(w_{t-1}).$$

In Worten bedeutet das folgendes:

Die Gewichte (Weights=Parameter) $w_t$ unseres Netzwerkes zum Zeitpunkt $t$ sind die bisherigen Gewichte $w_{t-1}$ minus dem Produkt aus Learning-Rate $\eta$ und Gradienten (der Ableitung) von der Lossfunktion $L$ an der Stelle der bisherigen Parameter $w_{t-1}$. 

Vorstellen kann man sich das ganze folgendermaßen:

* Man befindet sich irgendwo (zufällige Gewichte und somit Position am Anfang) in einem Gebirge (Loss-Landscape)
* Man möchte ins Tal (das *globale* Minimum finden)
* Es ist stark nebelig und man sieht nur, wie steil es an der aktuellen Position ist (Gradient bzw. negativer Gradient)
* Man bewegt sich einen kleinen Schritt (Learning Rate) in die Richtung des steilsten Abstiegs (Descent)
* Man wiederholt das ganze so lange, bis sich die Position nicht mehr wirklich ändert (Konvergenz)

![Gradient_Descent_Mountain_2](../resources/gradient_descent_mountain_2.png)

(von https://krishparekh.hashnode.dev/gradient-descent)

Man kann sich das auch vorstellen, wie wenn man einen Ball die Loss Landscape herunterrollen lässt (ohne Trägheit etc.), wie das folgende Beispiel zeigt.

![Gradient_Descent_Ball](../resources/gradient_descent_ball.jpg)

(von https://datamites.com/blog/what-is-a-gradient-descent/?srsltid=AfmBOoozS1Fi9nBr4PlpiG-m1LHiooWr6KzNQ6IWiiJ6rrMqSuGxWfAz)

> **Übung:** Warum ändert sich (bei passender Learning Rate) auf einmal die Position nicht mehr wirklich?

![Intuition_Gradient_Descent_Derivative](../resources/Gradient_Descent_Intiution_Derivative.png)

(von https://krishparekh.hashnode.dev/gradient-descent)

> **Übung:** Überlege, was bei so einer numerischen Methode schief gehen kann.

Es gibt 2 (bzw. 3) Dinge, die bei Optimieren mit Gradient Descent schief gehen können:
1. Wir landen in einem lokalen Minimum
2. Die Learning Rate passt nicht:
    * Die Learning Rate ist zu klein
    * Die Learning Rate ist zu groß

Erster Fall ist hier dargestellt.

![Gradient_Descent_Local_Minima](../resources/Gradient_Descent_Local_Minima.png)

(von https://nvsyashwanth.github.io/machinelearningmaster/understanding-gradient-descent/)

Es hängt also auch von der Startposition (wir starten mit zufälligen Gewichten) ab, wie gut das Netzwerk lernen kann. Wenn wir Pech haben, dann können wir keine gute Lösung finden.

Auch für die Learning Rates zeigen wir nun zwei mögliche (ungünstige) Fälle.

![Too_Small_Learning_Rate](../resources/Gradient_Descent_Too_Small_LR.png)

(von https://krishparekh.hashnode.dev/gradient-descent)

![Too_Large_Learning_Rate](../resources/Gradient_Descent_Too_Big_LR.png)

(von https://krishparekh.hashnode.dev/gradient-descent)

**Wie können wir die eben erwähnten Probleme lösen?**

Für eine zu kleine oder zu große Learning Rate ist die Lösung recht einfach: Wir müssen die Learning Rate verändern.

Sprich sollten wir das Gefühl haben, die Performance vom Modell schwankt sehr stark, dann sollten wir die Learning Rate reduzieren. Genauso sollten wir, falls wir das Gefühl haben, dass unser Modell zu langsam lernt und der Loss nach wie vor jede Iteration weniger wird, die Learning Rate (etwas) erhöhen.

Meistens liegt die Learning Rate im Hundertstel oder Tausendstel Bereich, sprich eine Standard Learning-Rate liegt oft im Bereich $0.001-0.01$.

**Hinweis:** Wir werden zu einem späteren Zeitpunkt sehen, wie wir den Lernprozess beobachten können (Spoiler: Wir lassen uns laufend aktuelle Werte vom Loss/Accuracy/etc. ausgeben) und somit beurteilen können, ob das Modell vernünftig lernt. Insbesondere werden wir uns das im Notebook bzgl. der *Trainingsmethode* ansehen.

**Frage:** Was, wenn wir in einem lokalem Minimum landen?

So ein Fall tritt wahrscheinlich bei jedem der praktischen Machine Learning Problemen auf. Auch, wenn es natürlich erwünscht wäre, in einem globalen Minimum zu landen, müssen wir uns meistens mit einem lokalen Minimum zufrieden geben. Dies hat folgende Gründe:
* Wir können nicht wissen, ob wir in einem lokalen oder globalen Optimum sind
* Sofern die Performance (Loss oder andere Metriken) gut genug ist, sind wir zufrieden
* Es gibt ein paar theoretische Resultate, bei denen gezeigt wird, dass in vielen Fällen bei komplizierten Machine Learning Tasks alle (lokalen) Minima gleich gut sind.

Falls wir doch das Gefühl haben, dass unser Modell viel schlechter performt, als es sollte, so können wir:
* Hyperparameter ändern (Learning Rate, Modellarchitektur)
* Mehr Daten verwenden (nicht immer verfügbar)
* Die Gewichte anders zufällig initialisieren (unüblich, nur "Experten" verändern diese Initialisierung, bringt nur in Spezialfällen was)
* Einen anderen Optimierungsalgorithmus, sprich einen anderen Optimizer verwenden.

**Hinweis:** Als Optimierer wird jener Algorithmus bezeichnet, der verwendet wird um zu lernen. In unserem Fall zBsp. (eine Variante von) Gradient Descent. Eine sehr bekannte Variante davon ist **Stochastic Gradient Descent** (SGD), welchen wir uns nun genauer ansehen wollen.

## Stochastic Gradient Descent

Der erste Optimizer den wir betrachten wollen ist der sogenannte *Stochastic Gradient Descent* (**SGD**) Algorithmus. 

Er ist quasi der "Standard" Algorithmus in PyTorch für die Optimierung, und adaptiert den bisher besprochenen (Vanilla) Gradient Descent Algorithmus nur leicht.

Sehen wir uns zuerst ein Bild an, welches den Stochastic Gradient Descent mit dem (Vanilla) Gradient Descent vergleicht.

![SGD_vs_GD](../resources/SGD_vs_GD.png)

(von https://www.geeksforgeeks.org/machine-learning/ml-stochastic-gradient-descent-sgd/)

Dieses Bild sieht jetzt vermutlich unintuitiv aus, weil die einzelnen Schritte des (Vanilla) Gradient Descent Algorithmus ja wesentlich besser aussehen. Diese Sichtweise ist teilweise richtig. Sehen wir uns mal die Details vom Stochastic Gradient Descent Algorithmus an.

Beim *SGD* ist die Update Rule leicht angepasst:
$$w_t = w_{t-1} - \eta \cdot \nabla L_I(w_{t-1}).$$

Der einzige Unterschied ist also, dass wir jetzt die Ableitung (den Gradient) von $L_I$ nehmen. Dabei bezeichnen wir mit $L_I$ die Loss-Funktion auf eine Teilmenge der Daten.

Sprich einfach gesagt macht Stochastic Gradient Descent nichts anderes als Gradient Descent, nur wird der Fehler und somit auch die Ableitung nur für ein paar wenige Datenpaare $(\hat y, y)$ berechnet anstatt für alle. Dabei werden die Datenpaare zufällig ausgewählt (stochastisch).

**Vorteile von SGD:**
* Schneller, weil die Ableitung für weniger Punkte berechnet werden muss
* Aufgrund von zufälliger Wahl der Datenpaare wird dem Optimierungsprozess etwas Stochastik ("Zufall") hinzugefügt, dadurch besteht die Chance, aus lokalen Minima auszubrechen
* Garantie, dass wir in Erwartung (im Mittel) das gleiche machen wie der (Vanilla) Gradient Descent Algorithmus (Sprich der Algorithmus macht quasi im Mittel Sinn).

**Nachteile von SGD:**
* Stochastik führt zu "instabilerem" Training (der Effekt ist aber nicht so drastisch, wie im obigen Bild dargestellt.)
* Schlechte Nachvollziehbarkeit/Reproduzierbarkeit

**Hinweis:** Den (Vanilla) Gradient Descent gibt es in PyTorch eigentlich nicht direkt. Es wird eigentlich immer *SGD* verwendeten, falls von Gradient Descent gesprochen wird.

**Wichtig:** Es gibt auch andere Optimierungsverfahren, bei denen die Formeln für die Updates aufwendiger sind, jedoch in manchen Fällen bessere Konvergenzverhalten liefern. Ein Beispiel dafür ist zum Beispiel **Adam** oder **Adagrad**, diese können genauso verwendet werden und funktionieren für viele Probleme besser (bzw. meistens nicht schlechter). Wir werden diese aber nicht in der Theorie behandeln.

![Meme_SGD](../resources/SGD_Local_Minima_Medal.jpg)

(von https://x.com/drob/status/1425468713017425923)

## Optimizer in PyTorch

Befassen wir uns nun damit, wie wir in Python (PyTorch) die Optimizer verwenden können und was sie bewirken.

Annahme, wir wollen die Funktion $f(w)=(w-3)^2$ minimieren. Wir sehen zwar relativ schnell, dass diese Funktion bei $x=3$ ihr Minimum erreicht, jedoch wollen wir das nun auch mit dem Gradient Descent Algorithmus zeigen.

**Hinweis:** Obiges Beispiel könnte man sich vorstellen, wie wenn wir die Funktion $f(x)=x$ haben mit Loss Funktion $g(x)=x^2$, sprich dem Mean Squarred Error. Das Label wär nun 3.

> **Übung:** Berechne die Werte von $w$ für die ersten 3 Iterationen mit (Vanilla) Gradient Descent, wobei als Startwert $w=0$ verwendet werden soll und als Learning Rate $\eta = 0.25$.

Als Hilfestellung wird hier die Update-Rule nochmal dargestellt:
$$w_t = w_{t-1} - \eta \cdot \nabla L(w_{t-1}).$$
Anders geschrieben ("Programmierschreibweise") heißt das
$$w \mathrel{-}= \eta \cdot \nabla L(w).$$
In unserem Fall (weil 1d) dann:
$$w \mathrel{-}= \eta \cdot f'(w).$$

**Dieses Beispiel in PyTorch:**

In [13]:
import torch

learning_rate = 0.25

# Parameter als Tensor mit Gradienten
x = torch.tensor([0.0], requires_grad=True)
optimizer = torch.optim.SGD([x], lr=learning_rate)

def f(x):
    return (x - 3)**2

loss = f(x)
print(f"Schritt 0: x = {x.tolist()}, f(x) = {loss.item():.4f}")
for i in range(10):
    optimizer.zero_grad()   # Gradienten zurücksetzen
    loss = f(x)
    loss.backward()         # Gradient berechnen
    optimizer.step()        # Update-Schritt
    print(f"Schritt {i+1}: x = {x.tolist()}, f(x) = {loss.item():.4f}")

Schritt 0: x = [0.0], f(x) = 9.0000
Schritt 1: x = [1.5], f(x) = 9.0000
Schritt 2: x = [2.25], f(x) = 2.2500
Schritt 3: x = [2.625], f(x) = 0.5625
Schritt 4: x = [2.8125], f(x) = 0.1406
Schritt 5: x = [2.90625], f(x) = 0.0352
Schritt 6: x = [2.953125], f(x) = 0.0088
Schritt 7: x = [2.9765625], f(x) = 0.0022
Schritt 8: x = [2.98828125], f(x) = 0.0005
Schritt 9: x = [2.994140625], f(x) = 0.0001
Schritt 10: x = [2.9970703125], f(x) = 0.0000


Hierbei sind folgende Punkte sehr wichtig in unserer Methode oben:

* `x = torch.tensor([0.0], requires_grad=True)` $x$ muss ein Tensor sein, bei dem der Gradient gespeichert wird. Startwert ist in unserem Fall $0.0$.
* `optimizer = torch.optim.SGD([x], lr=learning_rate)` Der Optimizer muss die Parameter des Modells kennen (in unserem Fall ist unser Modell nur $x$)
* `optimizer.zero_grad()` Am Anfang jeder Iteration muss der Gradient wieder aus dem Optimizer gelöscht werden (wir wollen ja wieder einen neuen Gradienten berechnen)
* `loss = f(x)` Diese Zeile würde bei einem neuronalen Netz dann die Prediction mit dem True Label vergleichen
* `loss.backward()` Berechnet die Gradienten bzgl. **aller** Parameter, die vorher dem Optimizer übergeben wurden
* `optimizer.step()` Führt das Update $w_t = w_{t-1} - \eta \cdot \nabla L(w_{t-1})$ durch

> **Übung:** Verändere den obigen Code so, dass wir für die drei vorigen Aufgaben vom Beginn das (bzw. ein) Minimum finden. Ändere dazu die Dimension (und ggf. den Startwert) von $x$.

Die verwendeten Funktionen waren:

$$L(w_1, w_2)=(4w_1+7w_2-20)^2$$
$$L(w_1, w_2, w_3)=\max (0, 2w_1+3w_2-4w_3)^2$$
$$L(w_1)=\lvert w_1 - e^{-w_1} \rvert$$

> **Übung:** Probiere auch die Optimizer *Adam* und *Adagrad* aus, indem du sie mit den Befehlen `from torch.optim import Adagrad, Adam` importierst und im Anschluss statt `SGD` verwendest.

## Loss Funktionen in PyTorch

Last but not least wollen wir uns noch die Loss-Funktionen in PyTorch ansehen.

Prinzipiell ist das Auswählen einer Loss-Funktion eine sehr kritische Sache, da wir im Laufe des Trainings versuchen, den Loss des Neuronalen Netzwerks zu reduzieren, idealerweise das (globale) Minimum davon zu finden. Es gibt jedoch für die Standard Probleme gängige Loss-Funktionen, die in vielen Fällen tadellos funktionieren. 

Diese beiden sind die bereits bekannten Funktionen:
* MSE (Mean Squared Error) für Regression
* (Binary) Cross-Entropy Loss für Klassifikation

Verwendet kann dies folgendermaßen werden.

In [ ]:
from torch.nn import CrossEntropyLoss, MSELoss

Dies sind jetzt noch Klassen, welche noch instanziert werden müssen.

In [ ]:
loss_fn = MSELoss()
loss_fn = CrossEntropyLoss()

Beides sind Funktionen, welche sich dann aufrufen lassen mit `loss_fn(input, target)`.

Auch, wenn der Output gleich ist wie oben beschrieben gibt es ein paar implementierungsspezifische Details, auf die man bei der Verwendung achten muss. Ein paar Dinge werden wir hier aufzählen. Allgemein findet man natürlich die Infos in der Dokumentation. So zum Beispiel auch für den [Cross-Entropy-Loss](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

Besonderheiten in der Implementierung:
* Wir erwarten uns beim Output Layer für eine Klassifikation normalerweise einen "Wahrscheinlichkeitsvektor", sprich die Summe muss 1 ergeben. Dies erreichen wir mit der Softmax Funktion am Schluss vom Netzwerk. In der Implementierung von PyTorch werden aber die Werte **vor der Softmax** Funktion erwartet
* Wir haben bisher gelernt, dass wir für die Klassifikation **One-Hot**-Vektoren für die Labels brauchen. Das stimmt in der Theorie, jedoch für die Implementierung reicht ein Integer Label aus.

> **Übung:** Warum reicht ein Integer Label aus, um den Loss zu berechnen? (Sprich warum kann sich PyTorch das Transformieren zu einem One-Hot Vektor sparen?)

**Hinweis:** (Nicht recht relevant für Test): Für eine Multiclass Klassifikation (wird bei uns nicht behandelt) reicht ein Index natürlich nicht mehr aus.

**Wichtig:** Solche Implementierungsdetails sollen beim Implementieren bekannt sein (oder beim Auftreten der Probleme zumindest nicht überraschen), jedoch ist es wichtiger, dass die zu Grunde liegende Theorie verstanden wird! 

## Vanishing and Exploding Gradient

Ok, nachdem wir jetzt wissen, wie ein Neuronales Netzwerk lernt, stellt man sich vielleicht die Frage, warum man nicht einfach ein riesiges (tiefes) Netzwerk verwendet. Immerhin kann man dann mit Gradient Descent oder verwandte/ähnliche Optimierer die (hoffentlich) beste Lösung finden und man hat eine gute Performance.

Die Idee ist zwar prinzipiell nicht falsch, jedoch funktioniert das in der Praxis nicht. Grund dafür ist der sogenannte **Vanishing Gradient** Effekt. 

### Vanishing Gradient

(Erstmals formalisiert/festgestellt von Prof. Sepp Hochreiter 1991).

Nachdem beim Gradient Descent die Ableitungen bezüglich der Gewichte berechnet werden, kommt natürlich auch die Kettenregel ins Spiel. Das wird bei zu tiefen Modellen oft ein Problem, da zum Beispiel

$$\frac{\partial L}{\partial w_1} = \frac{\partial L}{\partial a_n}\underbrace{\frac{\partial a_n}{\partial z_n}}_{=f'(a_n)} \frac{\partial z_n}{\partial a_{n-1}}\cdots \frac{\partial z_3}{\partial a_2}\underbrace{\frac{\partial a_2}{\partial z_2}}_{=f'(a_2)}\frac{\partial z_2}{\partial a_1}\underbrace{\frac{\partial a_1}{\partial z_1}}_{=f'(a_1)}\frac{\partial z_1}{\partial w_1}.$$

> **Übung:** Wie nennt man obige Regel beim Ableiten?

Hier stellt $f$ die Aktivierungsfunktion, somit $f'$ die Ableitung davon dar.

Warum ist das ein Problem?

Wenn wir uns die Ableitung von der Sigmoid Funktion $\sigma(x) = (1+e^{-x})^{-1}$ ansehen, dann erhalten wir
$$\sigma'(x) = \sigma(x) \cdot (1-\sigma(x)).$$

Diese Funktion hat als Maximalwert $0.25$. Mit der Kettenregel wird dieser Wert aber je nach Tiefe oft multipliziert.

![Sigmoid_Gradient](../resources/Sigmoid_plus_Derivative.png)

(eigene Abbildung)

Das bedeutet, wenn wir die Sigmoid Funktion als Aktivierungsfunktion verwenden und ein sehr tiefes Netzwerk verwenden, dann wird die Ableitung bezüglich der Gewichte am Anfang sehr klein (sie verschwindet - it vanishes).

Somit werden **extrem tiefe Modelle nicht mehr lernbar**, wenn die Sigmoid (oder vergleichbare) Aktivierungsfunktion verwendet wird. 

**Hinweis:** Die Aktivierungsfunktionen müssen natürlich nicht gleich sein bei jeder Schicht, aber mit jedem zusätzlichen Layer wird das Netzwerk tiefer und somit mit der Kettenregel ein weiterer Term (bzw. 2 weitere Terme) dazu multipliziert.

### Exploding Gradient

Das gleiche Problem kann passieren, wenn wir Aktivierungsfunktionen verwenden, bei denen die Ableitung größer als 1 ist. In solchen Fällen wird der Gradient sehr groß (er explodiert). Auch solche Netzwerke sind nicht lernbar.

> **Übung:** Warum sind Netzwerke, welche vom *Exploding-Gradient* betroffen sind, nicht lernbar?

### Gradient $\approx$ 1

Ideal ist natürlich eine Aktivierungsfunktion mit Werte der Ableitung in der Größenordnung 1. Solche Netzwerke können viel tiefer gemacht werden und leiden somit nicht unter den beiden oben genannten Problemen.

Ein sehr guter Kandidat: $\mathrm{ReLU}(x)$. Jedoch ist hier in vielen Fällen die Ableitung $0$.

**Hinweis:** In der Theorie kann natürlich ein Netzwerk bei genügend Iterationen/Epochen trainiert werden. Hier können die Gradienten dann zwar sehr sehr sehr klein werden, jedoch sind sie nicht $0$. In der Praxis sind die sehr kleinen Gradienten ein Problem, weil sie erstens den Trainingsprozess verlangsamen und andererseits ab einer gewissen Größe der Computer sie nicht mehr von der $0$ unterscheiden kann!

![Wait_Has_Been_Meme](../resources/Wait_Always_Has_Been.jpg)

(Eigene Grafik mit imgflip.com)